<a href="https://colab.research.google.com/github/whelfjhte79/BlindDateApp/blob/main/Face_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/dataset/Humans.zip -d /content/HumanFace/

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

In [ ]:
train_df = pd.read_csv('../content/faceData.csv', encoding= 'CP949')
print(train_df.columns.values)

In [ ]:
image = cv2.imread('/content/HumanFace/1 (103).jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
image_mask = []
imgPath = "/content/HumanFace/"
for i in os.listdir(imgPath):
    pic = os.path.join(imgPath + "/", i)
    image_mask.append(pic)

mask = pd.DataFrame()
mask["image"] = image_mask
mask